In [3]:
!pip install durable-rules
!pip install networkx

## Apresentação

*durable-rules*

Biblioteca do Python que facilita a construção de Sistemas Baseados em Regras (SBR) e lógica complexa. É particularmente útil para criar **Sistemas Especialistas**, fluxos de trabalho automatizados, e processamento de eventos em tempo real. A biblioteca se destaca por consiguir tratar regras  declarativas e por sua eficiência em lidar com grandes volumes de dados e eventos de maneira rápida e escalável.

### Componentes Chave do Durable-Rules

1. **Regras e Fatos**: No centro de durable-rules estão as regras, que são condições lógicas que, quando satisfeitas por fatos ou eventos, desencadeiam ações específicas. Fatos são unidades de informação que podem ser verificadas contra essas regras.

2. **Motores de Regras**: A biblioteca utiliza um motor de regras para avaliar os fatos à luz das regras definidas. Este motor pode processar regras complexas que envolvem múltiplas condições lógicas e relações entre diferentes tipos de dados.

3. **Contexto e Sessão**: O contexto é um armazenamento temporário onde os fatos são mantidos durante a avaliação das regras. Uma sessão dentro de durable-rules representa uma instância onde regras e fatos são processados.

### Funcionamento do Durable-Rules

O funcionamento de`durable-rules pode ser dividido em algumas etapas principais:

1. **Definição de Regras**: As regras são definidas em um formato declarativo, especificando condições e ações. As condições determinam quando uma regra é acionada, e as ações definem o que acontece quando a condição é verdadeira.

2. **Postagem de Fatos**: Os fatos são dados enviados ao motor de regras, que são analisados em relação às regras definidas. Um fato pode ser qualquer informação relevante, como dados de sensores, entradas de usuário, etc.

3. **Avaliação de Regras**: Quando um fato é postado, o motor de regras avalia todas as regras aplicáveis. Se as condições de uma regra são satisfeitas pelo fato, as ações correspondentes são executadas.

4. **Execução de Ações**: As ações podem incluir a modificação de fatos, a geração de novos eventos, ou a execução de qualquer código Python, como enviar notificações, iniciar outros processos, etc.

5. **Orientada a Eventos**: É altamente adequada para aplicações que necessitam de processamento e resposta em tempo real a eventos.

### Vantagens do Durable-Rules

- **Desempenho e Escalabilidade**: Projetado para alto desempenho, durable-rules pode processar milhões de mensagens por segundo, sendo ideal para aplicações que necessitam de análise rápida de grandes fluxos de dados.
- **Flexibilidade**: Permite a integração com outras tecnologias Python e pode ser usado em diversos cenários de aplicação, de sistemas IoT (Internet das Coisas) a complexos sistemas de monitoramento e alerta.
- **Facilidade de Uso**: A sintaxe declarativa simplifica a definição de regras complexas, tornando o código mais limpo e fácil de manter.

A biblioteca pode ajudar a criar jogos mais dinâmicos e interativos onde as reações e mudanças no ambiente do jogo são governadas por um conjunto de regras pré-definidas que podem ser dinamicamente ajustadas e expandidas.



# Aplicações em Games

## Coleta de itens

In [ ]:
#Importando  os itens do módulo durable.lang necessários para construção da base de conhecimento
from durable.lang import ruleset, when_all, post, m, assert_fact, c

# Lista para armazenar mensagens
mensagens = []

# Definindo as regras do jogo
with ruleset('game_rules'):
  try:
    # Regra para mostrar um item encontrado ao explorar uma floresta
    @when_all((m.event == 'explorar') & (m.location == 'floresta'))
    def encontrar_item_floresta(c):
        mensagens.append('Você encontrou uma espada na floresta!')

    # Regra para mostrar um item encontrado ao explorar uma  caverna
    @when_all((m.event == 'explorar') & (m.location == 'caverna'))
    def encontrar_item_caverna(c):
        mensagens.append('Você encontrou uma poção de cura na caverna!')

    # Regra para mostrar um item encontrado ao explorar uma uma masmorra
    @when_all((m.event == 'explorar') & (m.location == 'masmorra'))
    def encontrar_item_raro(c):
        mensagens.append('Você encontrou um amuleto da sorte na masmorra!')
  except Exception as e:
    print(f"Erro ao registrar o ruleset: {e}")


In [ ]:
# Função para simular a exploração do jogador
def explorar_localizacao(localizacao):
    assert_fact('game_rules', {'event': 'explorar', 'location': localizacao})

In [ ]:
explorar_localizacao('floresta')

In [ ]:
# Impressão das mensagens
for mensagem in mensagens:
    print(mensagem)

Você encontrou uma espada na floresta!


## Saudando o Jogador

In [ ]:
from durable.lang import ruleset, when_all, post, m, assert_fact, c
with ruleset('game'):
  try:
    # antecedente
    @when_all(m.subject == 'Player')
    def say_hello(c):
        # consequente
        print ('Hello {0}'.format(c.m.subject))
  except Exception as e:
    print(f"Erro ao registrar o ruleset: {e}")

In [ ]:
assert_fact('game',{'subject': 'Player'})

Hello Player


{'sid': '0', 'id': 'sid-0', '$s': 1}

## Comportamento Local
Neste exemplo, vamos simular um inimigo que pode ter diferentes comportamentos locais baseados em sua saúde e na distância entre ele e o jogador.

>Cenário:

>>No Estado de Alerta, ativado na MEF quando o jogador entra no raio de visão do inimigo, o inimigo irá atacar se estiver a uma distância curta do jogador.
Se o inimigo estiver a uma distância média, ele tentará se aproximar.
Se o inimigo estiver com pouca saúde, ele se afastará, independentemente da distância.

In [1]:
from durable.lang import ruleset, when_all, post, m, assert_fact, c

with ruleset('alerta'):
    # Regra para inimigo se afastar quando a saúde estiver baixa
    @when_all( m.health < 20)
    def fend(c):
        print(c.m)
        print('O inimigo está com pouca saúde e se afasta!')
    # Regra para inimigo atacar quando estiver perto do jogador
    @when_all(m.distance < 10)
    def attack(c):
        print(c.m)
        print('O inimigo ataca o jogador!')

    # Regra para inimigo se aproximar ou abordar o jogador se estiver a uma distância média
    @when_all((m.distance >= 10) & (m.distance < 30))
    def approach(c):
        print(c.m)
        print('O inimigo aborda o jogador.')

    # Regra para inimigo observar quando nenhuma das outras condições for atendida
    @when_all(+m.distance & +m.health)
    def observe(c):
        print('O inimigo observa o jogador.')


### Inferência

In [2]:
post('alerta', {'health': 15, 'distance': 5})

{'health': 15, 'distance': 5}
O inimigo está com pouca saúde e se afasta!


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [3]:
post('alerta', {'health': 30, 'distance': 25})

{'health': 30, 'distance': 25}
O inimigo aborda o jogador.


{'sid': '0', 'id': 'sid-0', '$s': 1}

In [4]:
post('alerta', {'health': 50, 'distance': 50})

O inimigo observa o jogador.


{'sid': '0', 'id': 'sid-0', '$s': 1}

### Sofisticando um pouco mais o comportamento do inimigo

In [1]:
from durable.lang import ruleset, when_all, post, m, assert_fact, c

with ruleset('alerta'):
    # Regra para inimigo se afastar quando a saúde estiver baixa, exceto se o jogador estiver muito longe (definido aqui como maior que 50 unidades de distância)
    @when_all((m.health < 20) & (m.distance <= 50))
    def fend(c):
        print(c.m)
        print('O inimigo está com pouca saúde e se afasta, pois o jogador não está longe!')

    # Regra para inimigo não fazer nada se a saúde estiver baixa e o jogador estiver muito longe
    @when_all((m.health < 20) & (m.distance > 50))
    def too_far_to_care(c):
        print(c.m)
        print('O inimigo está com pouca saúde, mas o jogador está muito longe para se preocupar.')

    # Regra para inimigo atacar quando estiver perto do jogador e tiver saúde boa
    @when_all((m.distance < 30) & (m.health >= 20))
    def attack(c):
        print(c.m)
        print('O inimigo com saúde boa ataca o jogador !')

    # Regra para inimigo se aproximar se estiver a uma distância média e saúde boa
    @when_all((m.distance >= 30) & (m.distance < 60) & (m.health >= 20))
    def approach(c):
        print(c.m)
        print('O inimigo aborda o jogador.')

    # Regra para inimigo observar quando nenhuma das outras condições for atendida
    @when_all(+m.distance & +m.health)
    def observe(c):
        print(c.m)
        print('O inimigo observa o jogador.')




### Inferência

In [2]:
post('alerta', {'health': 15, 'distance': 40})  # Deve fugir
post('alerta', {'health': 15, 'distance': 60})  # Jogador muito longe para se preocupar
post('alerta', {'health': 30, 'distance': 20})  # Deve atacar
post('alerta', {'health': 30, 'distance': 50})  # Deve se aproximar
post('alerta', {'health': 50, 'distance': 70})  # Deve observar

{'health': 15, 'distance': 40}
O inimigo está com pouca saúde e se afasta, pois o jogador não está longe!
{'health': 15, 'distance': 60}
O inimigo está com pouca saúde, mas o jogador está muito longe para se preocupar.
{'health': 30, 'distance': 20}
O inimigo com saúde boa ataca o jogador !
{'health': 30, 'distance': 50}
O inimigo aborda o jogador.
{'health': 50, 'distance': 70}
O inimigo observa o jogador.


{'sid': '0', 'id': 'sid-0', '$s': 1}

<h1>Pac-man</h1>

In [2]:
from durable.lang import ruleset, when_all, post, m, assert_fact, c
with ruleset('comportamento_fantasma'):

    @when_all((m.vulneravel == True) and (m.name == 'Blinky'))
    def fugir(c):
        print(c.m)
        print('Fantasma foge para o canto superior direito do mapa')

    @when_all((m.vulneravel == True) and (m.name == 'Inky'))
    def fugir(c):
        print(c.m)
        print('Fantasma foge para o canto inferior direito do mapa')
    
    @when_all((m.vulneravel == True) and (m.name == 'Pinky'))
    def fugir(c):
        print(c.m)
        print('Fantasma foge para o canto superior esquerdo do mapa')
    
    @when_all((m.vulneravel == True) and (m.name == 'Clyde'))
    def fugir(c):
        print(c.m)
        print('Fantasma foge para o canto inferior esquerdo do mapa')
    
    
    @when_all(m.name == 'Inky')
    def inky(c):
        print(c.m)
        print('')

    @when_all(m.name == 'Blinky')
    def blinky(c):
        print(c.m)
        print('')

    @when_all(m.name == 'Pinky')
    def pinky(c):
        print(c.m)
        print('')

    @when_all(m.name == 'Clyde')
    def clyde(c):
        print(c.m)
        print('')

In [3]:
post('comportamento_fantasma',{'vulneravel' : True, 'name':'Inky'})

{'vulneravel': True, 'name': 'Inky'}
Fantasma foge para o canto inferior direito do mapa


{'sid': '0', 'id': 'sid-0', '$s': 1}